This notebook is based largely on @mf1024's "Teach GPT-2 a sense of humour"
I've graciously borrowed much of his logic for fine tuning and generating from his implmenetation
Check out his notebook here https://github.com/mf1024/Transformers/blob/master/Teaching%20GPT-2%20a%20sense%20of%20humor.ipynb

In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, WarmupLinearSchedule
import numpy as np
import os

ModuleNotFoundError: No module named 'torch'

In [2]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)

In [4]:
def choose_top_n(probs, n=5):
    ind = np.argpartition(probs,-n)[-n:]
    top_probs = probs[ind]
    print(top_probs)
    top_probs = top_probs / np.sum(top_probs)
    choice = np.random.choice(n,1,p= top_probs)
    token_id = ind[choice][0]
    
    return int(token_id)

In [5]:
choose_top_n(np.array([0.99,0.6,0.6,0.91,0.9,0.6,0.88]))

[0.6  0.91 0.9  0.99 0.88]


5

In [6]:
FILE_PATH = os.path.join("data", "film_text.txt")

In [12]:
class ScriptDataset(Dataset):
    def __init__(self):
        super().__init__()
        # Note: It is probably better to get the tokenization of 
        # the script text done earlier. That way, we don't need to worry about it when going through
        # looping through for training. 
        assert os.path.isfile(FILE_PATH)
        directory, filename = os.path.split(FILE_PATH)
        
        with open(FILE_PATH, "r") as f:
            data = f.readlines()
            
        self.script_text  = []
        self.end_of_text_token = "<|endoftext|>"
        for i, script_entry in enumerate(data):
            script_str = f"{self.end_of_text_token}START:{script_entry}{self.end_of_text_token}"
            self.script_text.append(script_str)
        
        
        def __len__(self):
            return len(self.script_text)
        
        def __getitem__(self,item):
            return self.script_text[item]

In [7]:
from language_modelling import ScriptData

In [8]:
dataset = ScriptData(tokenizer= tokenizer, file_path= FILE_PATH )
script_loader = DataLoader(dataset,batch_size=1,shuffle=False)

In [10]:
BATCH_SIZE = 1
EPOCHS = 3
LEARNING_RATE = 1e-5
WARMUP_STEPS = 10000
MAX_SEQ_LEN = 550

In [11]:
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=WARMUP_STEPS, t_total = -1)
script_count = 0
sum_loss = 0.0
batch_count = 0

In [19]:
for epoch in range(EPOCHS):
    print(f"EPOCH {epoch} started" + '=' * 30)
    for idx,script in enumerate(script_loader):
        outputs = model(script, labels=script)
        
        loss, logits = outputs[:2]                        
        loss.backward()
        sum_loss = sum_loss + loss.detach().data
                       
        script_count = script_count + 1
        if script_count == BATCH_SIZE:
            joke_count = 0    
            batch_count += 1
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()
            
        if batch_count == 1000:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0

EPOCH 0 started==============================


KeyboardInterrupt: 